In [ ]:
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
zf = zipfile.ZipFile("/data/households.zip")
households = pd.read_csv(zf.open("Haushalte100m.csv"), delimiter=",", encoding="cp1252")

In [ ]:
households

In [ ]:
# read excel
translations = pd.read_excel("Data_Format_Census.xlsx", sheet_name="Translations")

In [ ]:
translations

In [ ]:
# tansfrom to dictionary with original as key and translation as value
translation_dict = translations.set_index("Original")["Translated"].to_dict()
# translations_dict = dict(zip(translations['Original'], translations['Translated']))

In [ ]:
pivot_df = households.pivot(
    index="Gitter_ID_100m", columns=["Merkmal", "Auspraegung_Text"], values="Anzahl"
)

instead of replacing nan with 0, exclude these cells

In [ ]:
# rename columns with trtanslation_dict
pivot_df = pivot_df.rename(columns=translation_dict)

In [ ]:
pivot_df

In [ ]:
# print unique columns
print(pivot_df.columns.get_level_values(0).unique())

In [ ]:
total = pivot_df["TOTAL"]["Total"].sum()

In [ ]:
pivot_df[["TOTAL", "HOUSEHOLD_LIFESTYLE"]]

In [ ]:
pivot_df["HOUSEHOLD_LIFESTYLE_DC"] = (
    pivot_df["HOUSEHOLD_LIFESTYLE"].sum(axis=1) / pivot_df["TOTAL"]["Total"]
)
pivot_df["HOUSEHOLD_SIZE_DC"] = (
    pivot_df["HOUSEHOLD_SIZE"].sum(axis=1) / pivot_df["TOTAL"]["Total"]
)
pivot_df["HOUSEHOLD_TYPE_DC"] = (
    pivot_df["HOUSEHOLD_TYPE"].sum(axis=1) / pivot_df["TOTAL"]["Total"]
)

In [ ]:
pivot_df[["TOTAL", "HOUSEHOLD_LIFESTYLE", "HOUSEHOLD_LIFESTYLE_DC"]]

In [ ]:
pivot_df[["TOTAL", "HOUSEHOLD_SIZE", "HOUSEHOLD_SIZE_DC"]]

In [ ]:
pivot_df[["TOTAL", "HOUSEHOLD_TYPE", "HOUSEHOLD_TYPE_DC"]]

In [ ]:
lifestyle = pivot_df["HOUSEHOLD_LIFESTYLE_DC"].sum().sum()
size = pivot_df["HOUSEHOLD_SIZE_DC"].sum().sum()
type = pivot_df["HOUSEHOLD_TYPE_DC"].sum().sum()

In [ ]:
print("life:", lifestyle / total * 100)
print("size:", size / total * 100)
print("type:", type / total * 100)

In [ ]:
data1 = pivot_df["HOUSEHOLD_LIFESTYLE_DC"]
data2 = pivot_df["HOUSEHOLD_SIZE_DC"]
data3 = pivot_df["HOUSEHOLD_TYPE_DC"]


counts1, bins1 = np.histogram(data1)
counts1 = counts1 / counts1.sum() * 100

counts2, bins2 = np.histogram(data2)
counts2 = counts2 / counts2.sum() * 100

counts3, bins3 = np.histogram(data3)
counts3 = counts3 / counts3.sum() * 100


plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(bins1[:-1], bins1, weights=counts1)
plt.xlabel("Lifestyle")
plt.ylabel("Percentage of total")
plt.ylim(0, 100)
plt.subplot(1, 3, 2)
plt.hist(bins2[:-1], bins2, weights=counts2)
plt.xlabel("Size")
plt.ylim(0, 100)
plt.subplot(1, 3, 3)
plt.hist(bins3[:-1], bins3, weights=counts3)
plt.xlabel("Type")
plt.ylim(0, 100)


plt.tight_layout()


plt.show()

In [ ]:
pivot_df.to_csv("Households_100m_pivot.csv", sep=",", encoding="cp1252")